@dataset{banda_juan_m_2020_3757272,
  author       = {Banda, Juan M. and
                  Tekumalla, Ramya and
                  Wang, Guanyu and
                  Yu, Jingyuan and
                  Liu, Tuo and
                  Ding, Yuning and
                  Artemova, Katya and
                  Tutubalinа, Elena and
                  Chowell, Gerardo},
  title        = {{A large-scale COVID-19 Twitter chatter dataset for 
                   open scientific research - an international
                   collaboration}},
  month        = may,
  year         = 2020,
  note         = {{This dataset will be updated bi-weekly at least 
                   with additional tweets, look at the github repo
                   for these updates. Release: We have standardized
                   the name of the resource to match our pre-print
                   manuscript and to not have to update it every
                   week.}},
  publisher    = {Zenodo},
  version      = {54.0},
  doi          = {10.5281/zenodo.3723939},
  url          = {https://doi.org/10.5281/zenodo.3723939}
}

In [ ]:
#The following extracting data function is follow from https://github.com/thepanacealab/covid19_twitter/blob/master/COVID_19_dataset_Tutorial.ipynb
from IPython.display import clear_output
# #Twarc
# !pip install twarc
# # Tweepy 3.8.0
# !pip install tweepy
# #Argparse 3.2
# !pip install argparse
# #Xtract 0.1 a3
# !pip install xtract
# #Wget 3.2
# !pip install wget
# clear_output()
import gzip
import shutil
import os
import wget
import csv
import linecache
from shutil import copyfile
import ipywidgets as widgets
import numpy as np
import pandas as pd

In [ ]:
# To avoid flooding gitlab, the data folder is not uploaded, so we need to created it first
datafolder = ".\\data\\"
try:
    os.mkdir(datafolder)
except OSError:
    print ("Creation of the directory %s failed" % datafolder)
else:
    print ("Successfully created the directory %s " % datafolder)

from datetime import timedelta, date
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)
#Extract all the daily tweets from 2020-03-22 to 2021-03-22
#The git respositry is https://github.com/GrimmNoters/covid19_twitter
#This is a forked version from the origin database
#Unfortunately the database only provide tweets that specified language from 2020-07-26 until now, so we skipped the first four months.
start_date = date(2020, 8, 1)
end_date = date(2021, 3, 23)
d= None
dataset_URL= None
readfileN= None
savefileN = None
for single_date in daterange(start_date, end_date):
    d = single_date.strftime("%Y-%m-%d")
    #Clean would only retrieve the origin tweets but ignore retweets
    dataset_URL = "https://github.com/GrimmNoters/covid19_twitter/blob/master/dailies/",d,"/",d,"_clean-dataset.tsv.gz?raw=true"
    dataset_URL=''.join(dataset_URL)
    #Downloads the dataset (compressed in a GZ format)
    #!wget dataset_URL -O date.tsv.gz
    readfileN = d,".tsv.gz"
    readfileN=''.join(readfileN)
    savefileN = ".\\data\\",d,".tsv"
    savefileN=''.join(savefileN)
    print(dataset_URL)
#     Examine if local has saved this date's data
    if(os.path.exists(savefileN)):
        continue
    wget.download(dataset_URL, out=readfileN)
    #Unzips the dataset and gets the TSV dataset
    with gzip.open(readfileN, 'rb') as f_in:
        with open(savefileN, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    #Deletes the compressed GZ file
    os.unlink(readfileN)
clear_output()

In [ ]:
#For our project we only use English
#Also due to api limit we only select random 100 users for each day.
import random
filtered_language = "en"
for single_date in daterange(start_date, end_date):
    date = single_date.strftime("%Y-%m-%d")
    readfileN = ".\\data\\",date,".tsv"
    readfileN=''.join(readfileN)
    savefileN = ".\\data\\",date,"-filtered.tsv"
    savefileN=''.join(savefileN)
    filtered_tw = list()
    current_line = 1
    tmp = list()
    with open(readfileN) as tsvfile:
        if(os.path.exists(savefileN)):
            continue
        tsvreader = csv.reader(tsvfile, delimiter="\t")
        
        if current_line == 1:
            tmp.append(linecache.getline(readfileN, current_line))
            
            for line in tsvreader:
                if line[3] == filtered_language:
                    filtered_tw.append(linecache.getline(readfileN, current_line))
                current_line += 1
        for key in random.sample(filtered_tw, 100):
            tmp.append(key)
        filtered_tw = tmp
    print(readfileN,'Filtered')
    with open(savefileN, 'w') as f_output:
        for item in filtered_tw:
            f_output.write(item)
#     os.remove(readfileN)
clear_output() 
    

In [ ]:
# import json
# import tweepy
# from tweepy import OAuthHandler
# #Create a JSON file unless existed
# if(not os.path.exists("api_keys.json")):
#     # Authenticate
#     CONSUMER_KEY = ""
#     CONSUMER_SECRET_KEY = ""
#     ACCESS_TOKEN_KEY = ""
#     ACCESS_TOKEN_SECRET_KEY = ""

#     #Creates a JSON Files with the API credentials
#     with open('api_keys.json', 'w') as outfile:
#         json.dump({
#         "consumer_key":CONSUMER_KEY,
#         "consumer_secret":CONSUMER_SECRET_KEY,
#         "access_token":ACCESS_TOKEN_KEY,
#         "access_token_secret": ACCESS_TOKEN_SECRET_KEY
#      }, outfile)

In [ ]:
# # The lines below are just to test if the twitter credentials are correct
# # Authenticate
# auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET_KEY)

# api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# if (not api):
#    print ("Can't Authenticate")
#    sys.exit(-1)

In [ ]:
#!wget https://raw.githubusercontent.com/thepanacealab/SMMT/master/data_acquisition/get_metadata.py -O get_metadata.py

In [ ]:
#pip install easydict

In [ ]:
#     args = easydict.EasyDict({
#         "inputfile": "",
#         "outputfile": "",
#         "keyfile": "",
#         "idcolumn": None,
#         "mode": None,
#     })
from imp import reload
import get_metadata
reload(get_metadata)

In [ ]:
# Hydrating the data
folder = "data"
keyfile = "api_keys.json"
for single_date in daterange(start_date, end_date):
    date = single_date.strftime("%Y-%m-%d")
    infile = ".\\data\\",date,"-filtered.tsv"
    infile=''.join(infile)
    outfile = date,"-hydrated"
    outfile=''.join(outfile)
    get_metadata.hydrating(infile,folder, outfile, keyfile, None, None)
    clear_output() 

In [ ]:
# Remove temp files
# Unfortunately i should not remove the json file since it can be used later when combining daily to monthly
# but so far I have reach the limit of Twitter api request so I could not retrieve the files again and recreate the json files.
for single_date in daterange(start_date, end_date):
    date = single_date.strftime("%Y-%m-%d")
    file = date,"-hydrated"
    file=''.join(file)
    if(os.path.exists(file)):
        os.remove(file)
    file = file,"_short.json"
    file=''.join(file)
    if(os.path.exists(file)):
        os.remove(file)

In [ ]:
# Extracted dates
import datetime
from datetime import timedelta, date
def end_of_month(dt):
    todays_month = dt.month
    tomorrows_month = (dt + datetime.timedelta(days=1)).month
    return True if tomorrows_month != todays_month else False
def start_of_month(dt):
    return True if dt.day == 1 else False
start_date = date(2020, 8, 1)
end_date = date(2021, 3, 23)
file_Date = [[]]*8
i = 0
for single_date in daterange(start_date, end_date):
    if(start_of_month(single_date)):
        file_Date[i] = [single_date.strftime("%Y-%m-%d")]
    else:
        file_Date[i].append(single_date.strftime("%Y-%m-%d"))
#     file_Date[i].append([single_date.strftime("%Y-%m-%d")])
    if(end_of_month(single_date)):
        i=i+1

In [ ]:
# Merge all data of the same month
outF = ""
import pandas as pd
for i in range(len(file_Date)):
    outF = ''.join((".\\data\\",file_Date[i][0][:7]+"_hydrated.csv"))
    combined_csv = pd.concat([pd.read_csv(''.join((".\\data\\",one_day,"-hydrated.csv"))) for one_day in file_Date[i]])
    combined_csv.to_csv(outF, index=False, encoding='utf-8')